In [121]:
# Packages
import numpy as np
import sklearn 
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import odeint
from scipy.optimize import minimize
from utility_code.utility import utils

In [122]:
#import csvs
train = pd.read_csv("ucla2020-cs145-covid19-prediction/train.csv")
test = pd.read_csv("ucla2020-cs145-covid19-prediction/test.csv")
graph = pd.read_csv("ucla2020-cs145-covid19-prediction/graph.csv")

supp = pd.read_csv("data-test/raw_data_test.csv", skiprows=2, thousands=',')
supp = supp[supp['Location'].isin(train['Province_State'])]
supp['Population'] = supp['Number of COVID-19 Cases'].divide(supp['COVID-19 Cases per 1,000,000 Population']) * 1e6

states = pd.Series.unique(train['Province_State'])
num_states = len(states)

In [178]:
#only look at the features in features list
features = ['Confirmed','Deaths']
num_features = len(features)

#stratify by state (into state dictionary)
statesdata = {}
for s in states:
    statesdata[s] = train.loc[train['Province_State'] == s,features]

WINDOW_SIZE = 3
#WINDOW_SIZE = 5
state_feature_indices = utils.get_column_indices(statesdata['California'],features)

#append the feature spaces from the W days prior (where W is the window length)
new_features = []
for day in range(WINDOW_SIZE):
    for f in features:
        new_features.append(f + "(-"+ str(WINDOW_SIZE-day) + " days)")
all_new_features = new_features + features


In [179]:
# x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
# y = knndata[['Confirmed', 'Deaths']]

In [180]:
# toguess = 1
# trainx = x.head(len(x)-toguess)
# testx = x.tail(toguess)
# trainy = y.head(len(y)-toguess)
# testy = y.tail(toguess)

In [195]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
#from sklearn.ensemble import StackingRegressor
#estimators = [('ridge', Ridge(random_state = 123)), ('lasso', LassoCV(random_state=445)), 
              #('svr', SVR(kernel='linear',C=1, gamma=1e-6))]
#reg = MultiOutputRegressor(estimator=Ridge(alpha=1.3242529, tol = 0.00125, random_state=123))
reg = MultiOutputRegressor(estimator=ElasticNet(max_iter=2000, l1_ratio=0.1))
#reg = MultiOutputRegressor(estimator=Ridge(random_state=123))
#from sklearn.svm import LinearSVR
#reg = MultiOutputRegressor(estimator=LinearSVR(max_iter=2000))
# reg.fit(trainx, trainy)
# reg.predict(testx)

In [196]:
# states

# days_to_predict = 26
# for d in range(days_to_predict):
#     x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
#     y = knndata[['Confirmed', 'Deaths']]
#     toguess = 1
#     trainx = x.head(len(x))
#     trainy = y.head(len(y))
#     # testy = y.tail(toguess)

#     reg.fit(trainx, trainy)
    

#     ftrs = knndata.drop(columns=knndata.columns[[0,1]]).tail(1)
#     #ftrs.drop(columns=knndata.columns[[0,1]])
   
#     ftrs.columns = knndata.columns[0:2*WINDOW_SIZE]
    
#     new = reg.predict(ftrs)
#     ftrs = np.append(ftrs, new)
#     ftrs = ftrs.astype(int)
#     if d==0: print(ftrs)
#     knndata = knndata.append(dict(zip(knndata.columns, ftrs)), ignore_index=True)
    
#     # append to knndata
#     if d == 0: 
#         print(knndata)
# done = knndata.tail(days_to_predict)
# done = done[['Confirmed', 'Deaths']]
# print(done)

In [197]:
proj = {}

for s in states:
    
    a = statesdata[s]

    #fill the knn data using days from training set
    knndata = pd.DataFrame(columns = all_new_features)
    num_training_days = len(statesdata['California'])

    #fill the table
    for d in range(WINDOW_SIZE,num_training_days):
        knndata_row_index = knndata.shape[0]
        knn_row = utils.flatten_dataframe(a,slice(d-WINDOW_SIZE,d+1), state_feature_indices)
        utils.dataframe_append_row(knndata,knn_row,s,d)   
    

    days_to_predict = 26
    for d in range(days_to_predict):
        #x = knndata.drop(columns=features)
        #y = knndata.drop(columns=features)
        x = knndata.drop(['Confirmed', 'Deaths'], axis = 1)
        y = knndata[['Confirmed', 'Deaths']]
        toguess = 1
        trainx = x.head(len(x))
        trainy = y.head(len(y))
        # testy = y.tail(toguess)
        #reg = MultiOutputRegressor(estimator=Ridge(random_state=123, max_iter=1500))
        reg.fit(trainx, trainy)

        #rmv = [i for i in range(num_features)]
        #ftrs = knndata.drop(columns=knndata.columns[rmv]).tail(1)
        ftrs = knndata.drop(columns=knndata.columns[[0,1]]).tail(1)
        #ftrs.drop(columns=knndata.columns[[0,1]])

        ftrs.columns = knndata.columns[0:num_features*WINDOW_SIZE]

        new = reg.predict(ftrs)
        ftrs = np.append(ftrs, new)
        ftrs = ftrs.astype(int)
        #if d==0: print(ftrs)
        knndata = knndata.append(dict(zip(knndata.columns, ftrs)), ignore_index=True)

        # append to knndata
        #if d == 0: 
            #print(knndata)
    done = knndata.tail(days_to_predict)
    done = done[['Confirmed', 'Deaths']]
    #print(done)
    proj[s] = done

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 39210815.31017768, tolerance: 21269763.580464747
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11934.705338241367, tolerance: 5006.691290647483
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40100360.11017656, tolerance: 21928709.351774286
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incre

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 56411883.52421477, tolerance: 37463458.79032152
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15193.67715594287, tolerance: 8486.179329746838
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 57196974.359937295, tolerance: 38568700.01973334
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increas

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 99101.43930876043, tolerance: 66794.18661730769
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 49.1502342253437, tolerance: 1.7249608974358976
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 102823.94941293618, tolerance: 69530.13284840765
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increas

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 221555224.99580327, tolerance: 99174411.58474267
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 83110.55298474108, tolerance: 51480.1491477707
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 223341136.8346414, tolerance: 100191698.486669
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase 

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12755907.91485764, tolerance: 8311017.219170701
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1418.526795209366, tolerance: 1310.203057324841
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12847187.495678311, tolerance: 8503887.918235444
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increas

...

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13141.222401415302, tolerance: 8140.864883974359
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13198.63781935988, tolerance: 8257.234332484077
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13255.443971537234, tolerance: 8374.493473417722
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increa

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4235.365368290019, tolerance: 2682.8776683544306
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13602238.201557793, tolerance: 9546211.247767296
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4247.643474171866, tolerance: 2740.999389937107
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increa

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 328163.1623706219, tolerance: 215187.54666878982
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 319.6397349201127, tolerance: 59.16410063694268
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 333280.9947298032, tolerance: 223001.994593038
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11688272.840238743, tolerance: 9916468.783350326
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2174.9229078562917, tolerance: 1263.935305882353
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11782038.976292606, tolerance: 10203228.168704547
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to incr

/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40136.46975508968, tolerance: 26640.642835897437
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 76.91638679141579, tolerance: 1.7662173076923078
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40759.75764850023, tolerance: 27457.69769299363
  positive)
/Users/Aditya/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increa

In [198]:
order = test.loc[0:49,'Province_State']
test

# format submission
conf = []
dead = []
fid = 0
for i in range(days_to_predict):
    for j in order:
        projection = proj[j].iloc[i]
        print(j, 'day', i)
        conf.append(int(projection['Confirmed']))
        dead.append(int(projection['Deaths']))
        print(fid)
        fid+=1
    
    

Alabama day 0
0
Alaska day 0
1
Arizona day 0
2
Arkansas day 0
3
California day 0
4
Colorado day 0
5
Connecticut day 0
6
Delaware day 0
7
Florida day 0
8
Georgia day 0
9
Hawaii day 0
10
Idaho day 0
11
Illinois day 0
12
Indiana day 0
13
Iowa day 0
14
Kansas day 0
15
Kentucky day 0
16
Louisiana day 0
17
Maine day 0
18
Maryland day 0
19
Massachusetts day 0
20
Michigan day 0
21
Minnesota day 0
22
Mississippi day 0
23
Missouri day 0
24
Montana day 0
25
Nebraska day 0
26
Nevada day 0
27
New Hampshire day 0
28
New Jersey day 0
29
New Mexico day 0
30
New York day 0
31
North Carolina day 0
32
North Dakota day 0
33
Ohio day 0
34
Oklahoma day 0
35
Oregon day 0
36
Pennsylvania day 0
37
Rhode Island day 0
38
South Carolina day 0
39
South Dakota day 0
40
Tennessee day 0
41
Texas day 0
42
Utah day 0
43
Vermont day 0
44
Virginia day 0
45
Washington day 0
46
West Virginia day 0
47
Wisconsin day 0
48
Wyoming day 0
49
Alabama day 1
50
Alaska day 1
51
Arizona day 1
52
Arkansas day 1
53
California day 1
54


Arizona day 11
552
Arkansas day 11
553
California day 11
554
Colorado day 11
555
Connecticut day 11
556
Delaware day 11
557
Florida day 11
558
Georgia day 11
559
Hawaii day 11
560
Idaho day 11
561
Illinois day 11
562
Indiana day 11
563
Iowa day 11
564
Kansas day 11
565
Kentucky day 11
566
Louisiana day 11
567
Maine day 11
568
Maryland day 11
569
Massachusetts day 11
570
Michigan day 11
571
Minnesota day 11
572
Mississippi day 11
573
Missouri day 11
574
Montana day 11
575
Nebraska day 11
576
Nevada day 11
577
New Hampshire day 11
578
New Jersey day 11
579
New Mexico day 11
580
New York day 11
581
North Carolina day 11
582
North Dakota day 11
583
Ohio day 11
584
Oklahoma day 11
585
Oregon day 11
586
Pennsylvania day 11
587
Rhode Island day 11
588
South Carolina day 11
589
South Dakota day 11
590
Tennessee day 11
591
Texas day 11
592
Utah day 11
593
Vermont day 11
594
Virginia day 11
595
Washington day 11
596
West Virginia day 11
597
Wisconsin day 11
598
Wyoming day 11
599
Alabama day 12


Alaska day 21
1051
Arizona day 21
1052
Arkansas day 21
1053
California day 21
1054
Colorado day 21
1055
Connecticut day 21
1056
Delaware day 21
1057
Florida day 21
1058
Georgia day 21
1059
Hawaii day 21
1060
Idaho day 21
1061
Illinois day 21
1062
Indiana day 21
1063
Iowa day 21
1064
Kansas day 21
1065
Kentucky day 21
1066
Louisiana day 21
1067
Maine day 21
1068
Maryland day 21
1069
Massachusetts day 21
1070
Michigan day 21
1071
Minnesota day 21
1072
Mississippi day 21
1073
Missouri day 21
1074
Montana day 21
1075
Nebraska day 21
1076
Nevada day 21
1077
New Hampshire day 21
1078
New Jersey day 21
1079
New Mexico day 21
1080
New York day 21
1081
North Carolina day 21
1082
North Dakota day 21
1083
Ohio day 21
1084
Oklahoma day 21
1085
Oregon day 21
1086
Pennsylvania day 21
1087
Rhode Island day 21
1088
South Carolina day 21
1089
South Dakota day 21
1090
Tennessee day 21
1091
Texas day 21
1092
Utah day 21
1093
Vermont day 21
1094
Virginia day 21
1095
Washington day 21
1096
West Virginia da

In [199]:
test['Confirmed'] = conf
test['Deaths'] = dead

In [200]:
submission = test.drop(columns=['Date', 'Province_State'])
print(submission)

      ForecastID  Confirmed  Deaths
0              0     128493    2208
1              1       5372      37
2              2     203144    5086
3              3      62181     812
4              4     728523   13231
...          ...        ...     ...
1295        1295     147759    2869
1296        1296      97223    2206
1297        1297      15203     274
1298        1298     101870    1282
1299        1299       5292      37

[1300 rows x 3 columns]


In [131]:
#submission.to_csv("team31-dec1-test1.csv", index=False, header=True)

In [14]:
trainrd2 = pd.read_csv("train_round2.csv")
trainrd2

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
0,0,Alabama,04-12-2020,3563,93,NaN,3470.0,75.988020,21583.0,437.0,2.610160,460.300152,12.264945
1,1,Alaska,04-12-2020,272,8,66.0,264.0,45.504049,8038.0,31.0,2.941176,1344.711576,11.397059
2,2,Arizona,04-12-2020,3542,115,NaN,3427.0,48.662422,42109.0,NaN,3.246753,578.522286,NaN
3,3,Arkansas,04-12-2020,1280,27,367.0,1253.0,49.439423,19722.0,130.0,2.109375,761.753354,10.156250
4,4,California,04-12-2020,22795,640,NaN,22155.0,58.137726,190328.0,5234.0,2.812020,485.423869,22.961176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11245,11245,Virginia,11-22-2020,217796,3938,23133.0,190725.0,2551.643315,NaN,NaN,NaN,36540.179923,NaN
11246,11246,Washington,11-22-2020,141260,2619,NaN,138641.0,1855.049047,NaN,NaN,NaN,38009.293105,NaN
11247,11247,West Virginia,11-22-2020,40478,662,26476.0,13340.0,2258.631686,NaN,NaN,NaN,57303.837241,NaN
11248,11248,Wisconsin,11-22-2020,376238,3150,276574.0,96514.0,6461.868009,NaN,NaN,NaN,71493.399496,NaN


In [15]:
datestofind = trainrd2['Date'] == '09-01-2020'
start = datestofind[datestofind == True].index[0]
datestofind2 = trainrd2['Date'] == '09-26-2020'
end = datestofind[datestofind == True].index[49]

validation = trainrd2.iloc[start:end,:]
validation

,ID,Province_State,Date,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
7100,7100,Alabama,09-01-2020,127616,2200,48028.0,77388.0,2602.716398,958920.0,NaN,1.723922,19557.083814,NaN
7101,7101,Alaska,09-01-2020,5297,39,2246.0,3012.0,724.083959,374255.0,NaN,0.736266,51159.532223,NaN
7102,7102,Arizona,09-01-2020,202342,5044,30841.0,166457.0,2779.912998,1208990.0,NaN,2.492809,16609.932767,NaN
7103,7103,Arkansas,09-01-2020,61497,814,55647.0,5036.0,2037.806299,731025.0,NaN,1.323642,24223.740177,NaN
7104,7104,California,09-01-2020,715617,13150,NaN,702467.0,1811.128167,11470696.0,NaN,1.837575,29030.753344,NaN
7105,7105,Colorado,09-01-2020,57755,1946,5953.0,49856.0,1002.911055,1009970.0,NaN,3.369405,17538.050017,NaN
7106,7106,Connecticut,09-01-2020,53006,4466,8967.0,39573.0,1486.724631,1192338.0,NaN,8.425461,33442.973876,NaN
7107,7107,Delaware,09-01-2020,17535,605,9419.0,7511.0,1800.744328,238423.0,NaN,3.450242,24484.680066,NaN
7108,7108,Florida,09-01-2020,631040,11374,NaN,619666.0,2938.112148,4675866.0,NaN,1.802421,21770.757320,NaN
7109,7109,Georgia,09-01-2020,272697,5733,NaN,266964.0,2568.391596,2368498.0,NaN,2.102333,22307.654127,NaN


In [12]:
def MAPE(pred, valid):
    pred = pred.reset_index()
    valid = valid.reset_index()
    pred = pred.astype('int64')
    valid = valid.astype('int64')
    v = pred.subtract(valid)
    v = v.divide(valid)
    v = v.abs()
    v = v.sum(axis = 0)
    #v = v[0]+v[1]+v[2]
    n = len(pred)
    return v/n

In [38]:
bestsub = pd.read_csv("team31-nov29-2.csv")

In [39]:
MAPE(bestsub[['Confirmed', 'Deaths']], validation[['Confirmed', 'Deaths']])

index        0.037565
Confirmed    0.000080
Deaths       0.000178
dtype: float64

In [201]:
MAPE(test[['Confirmed', 'Deaths']], validation[['Confirmed', 'Deaths']])

index        0.037565
Confirmed    0.000298
Deaths       0.000232
dtype: float64